# LSTM Binary Multiclass with CICIDS2017

If you are missing the pickle, try running the *Pickle-CICIDS2017.ipynb* Notebook.

In [1]:
from datetime import datetime
import json
import os
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

## Data loading and prep

As we've pickled the normalized and encoded dataset, we only need to load these pickles to get the Pandas DataFrames back.  
**Hint**: If you miss the pickles, go ahead and run the notebook named *Pickle-NSL-KDD.ipynb*

In [2]:
def load_df(filename):
    filepath = os.path.join('CICIDS2017', filename+'.pkl')
    return pd.read_pickle(filepath)

In [3]:
cic_train_data = load_df('cic_train_data')
cic_test_data = load_df('cic_test_data')
cic_train_labels = load_df('cic_train_labels')
cic_test_labels = load_df('cic_test_labels')

In [4]:
cic_train_data.tail()

,source_port,destination_port,protocol,flow_duration,total_fwd_packets,total_backward_packets,total_length_of_fwd_packets,total_length_of_bwd_packets,fwd_packet_length_max,fwd_packet_length_min,fwd_packet_length_mean,fwd_packet_length_std,bwd_packet_length_max,bwd_packet_length_min,bwd_packet_length_mean,bwd_packet_length_std,flow_bytes_per_s,flow_packets_per_s,flow_iat_mean,flow_iat_std,flow_iat_max,flow_iat_min,fwd_iat_total,fwd_iat_mean,fwd_iat_std,fwd_iat_max,fwd_iat_min,bwd_iat_total,bwd_iat_mean,bwd_iat_std,bwd_iat_max,bwd_iat_min,fwd_psh_flags,bwd_psh_flags,fwd_urg_flags,bwd_urg_flags,fwd_header_length,bwd_header_length,fwd_packets_per_s,bwd_packets_per_s,min_packet_length,max_packet_length,packet_length_mean,packet_length_std,packet_length_variance,fin_flag_count,syn_flag_count,rst_flag_count,psh_flag_count,ack_flag_count,urg_flag_count,cwe_flag_count,ece_flag_count,down_per_up_ratio,average_packet_size,avg_fwd_segment_size,avg_bwd_segment_size,fwd_header_length.1,fwd_avg_bytes_per_bulk,fwd_avg_packets_per_bulk,fwd_avg_bulk_rate,bwd_avg_bytes_per_bulk,bwd_avg_packets_per_bulk,bwd_avg_bulk_rate,subflow_fwd_packets,subflow_fwd_bytes,subflow_bwd_packets,subflow_bwd_bytes,init_win_bytes_forward,init_win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,active_mean,active_std,active_max,active_min,idle_mean,idle_std,idle_max,idle_min,source_ip_o1,source_ip_o2,source_ip_o3,source_ip_o4,destination_ip_o1,destination_ip_o2,destination_ip_o3,destination_ip_o4,external_ip_o1,external_ip_o2,external_ip_o3,external_ip_o4
1839977,0.455848,0.000809,1.0,0.000002,0.000005,0.000007,0.000029,2.532599e-07,0.001410,0.015264,0.007491,0.0,0.004250,0.028660,0.018990,0.0,0.112415,0.403902,6.027778e-07,9.062020e-07,0.000001,1.416667e-07,2.500000e-08,2.500000e-08,0.0,2.500000e-08,9.166666e-08,4.083333e-07,4.083333e-07,0.0,4.083333e-07,4.083333e-07,0.0,0.0,0.0,0.0,0.999856,0.994592,0.003252,0.004878,0.024171,0.003344,0.016241,0.005955,0.000035,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.009259,0.017402,0.007491,0.018990,0.999856,0.0,0.0,0.0,0.0,0.0,0.0,0.000005,0.000029,0.000007,2.532599e-07,0.0,0.0,0.000005,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.86036,0.658824,0.039216,0.07451,0.751969,0.658824,0.039216,0.011765,0.0,0.0,0.0,0.0
1839978,0.093996,0.000809,1.0,0.000521,0.000005,0.000007,0.000037,2.502086e-07,0.001813,0.019625,0.009632,0.0,0.004199,0.028315,0.018761,0.0,0.111923,0.400013,1.736833e-04,4.255664e-04,0.000521,1.250000e-07,8.333333e-09,8.333333e-09,0.0,8.333333e-09,7.500000e-08,3.333333e-08,3.333333e-08,0.0,3.333333e-08,3.333333e-08,0.0,0.0,0.0,0.0,0.999856,0.994592,0.000011,0.000016,0.031077,0.003304,0.017920,0.004591,0.000021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.009259,0.019199,0.009632,0.018761,0.999856,0.0,0.0,0.0,0.0,0.0,0.0,0.000005,0.000037,0.000007,2.502086e-07,0.0,0.0,0.000005,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.86036,0.658824,0.039216,0.07451,0.751969,0.658824,0.039216,0.011765,0.0,0.0,0.0,0.0
1839979,0.798459,0.000809,1.0,0.000510,0.000005,0.000007,0.000037,3.966722e-07,0.001813,0.019625,0.009632,0.0,0.006656,0.044890,0.029744,0.0,0.111924,0.400013,1.699778e-04,4.164739e-04,0.000510,1.416667e-07,2.500000e-08,2.500000e-08,0.0,2.500000e-08,9.166666e-08,2.500000e-08,2.500000e-08,0.0,2.500000e-08,2.500000e-08,0.0,0.0,0.0,0.0,0.999856,0.994592,0.000011,0.000016,0.031077,0.005238,0.023673,0.010546,0.000111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.009259,0.025364,0.009632,0.029744,0.999856,0.0,0.0,0.0,0.0,0.0,0.0,0.000005,0.000037,0.000007,3.966722e-07,0.0,0.0,0.000005,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.86036,0.658824,0.039216,0.07451,0.751969,0.658824,0.039216,0.011765,0.0,0.0,0.0,0.0
1839980,0.366766,0.000809,1.0,0.000512,0.000005,0.000007,0.000043,3.356457e-07,0.002095,0.022678,0.011130,0.0,0.005632,0.037983,0.025168,0.0,0.111923,0.400013,1.705889e-04,4.179615e-04,0.000512,1.416667e-07,2.500000e-08,2.500000e-08,0.0,2.500000e-08,9.166666e-08,3.333333e-08,3.333333e-08,0.0,3.333333e-08,3.333333e-08,0.0,0.0,0.0,0.0,0.999856,0.994592,0.000011,0.00

We only need 6 features, so we create a new DF that only holds them.  
The mapping is as follows:  

| NSL-KDD field | CICIDS2017 field |
|---------------|---------------------|
| duration | flow_duration |
| protocol_type | protocol |
| src_bytes | total_fwd_packets |
| dst_bytes | total_backward_packets |
| count | flow_packets_per_s |
| srv_count | destination_port |


In [5]:
fields = ['flow_duration', 'protocol', 'total_fwd_packets', 'total_backward_packets','flow_packets_per_s','destination_port']

cic_train_data = cic_train_data.filter(fields, axis=1) 
cic_test_data = cic_test_data.filter(fields, axis=1)

cic_train_data.tail()

,flow_duration,protocol,total_fwd_packets,total_backward_packets,flow_packets_per_s,destination_port
1839977,0.000002,1.0,0.000005,0.000007,0.403902,0.000809
1839978,0.000521,1.0,0.000005,0.000007,0.400013,0.000809
1839979,0.000510,1.0,0.000005,0.000007,0.400013,0.000809
1839980,0.000512,1.0,0.000005,0.000007,0.400013,0.000809
1839981,0.000523,1.0,0.000005,0.000007,0.400013,0.000809


## Label Translation

As we are doing binary classification, we only need to know if the entry is normal/benign (0) or malicious (1)

In [6]:
with open(os.path.join('CICIDS2017','cic_label_wordindex.json')) as json_in:
    data = json.load(json_in)
    print(data)
    normal_index = data['benign']

{'benign': 1, 'ftppatator': 2, 'sshpatator': 3, 'dosslowloris': 4, 'dosslowhttptest': 5, 'doshulk': 6, 'dosgoldeneye': 7, 'heartbleed': 8, 'bruteforce': 9, 'xss': 10, 'sqlinjection': 11, 'infiltration': 12, 'bot': 13, 'portscan': 14, 'ddos': 15}


In [7]:
def f(x):
    return 0 if x == normal_index else 1
f = np.vectorize(f)

In [8]:
cic_train_labels.head()

,label,label_encoded
0,BENIGN,1
1,BENIGN,1
2,BENIGN,1
3,BENIGN,1
4,BENIGN,1


In [9]:
# We only want to know if it's benign or not, so we switch to 0 or 1
cic_train_labels = f(cic_train_labels['label_encoded'].values)
cic_test_labels = f(cic_test_labels['label_encoded'].values)

In [10]:
cic_train_labels[:5]

array([0, 0, 0, 0, 0])

In [11]:
print("Training Set Size:\t",len(cic_train_data))
print("Training Label Size:\t",len(cic_train_labels))
print("Test Set Size:\t\t",len(cic_test_data))
print("Test Label Size:\t",len(cic_test_labels))

Training Set Size:	 1839982
Training Label Size:	 1839982
Test Set Size:		 990761
Test Label Size:	 990761


## Runtime Preqs

Let's go ahead and set some crucial parameters for the learning process

In [12]:
epochs = 50
batch_size = 32
no_of_classes = len(np.unique(np.concatenate((cic_train_labels, cic_test_labels))))

In [13]:
run_date = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
runtype_name = 'cicids-lstm-2multiclass-b{}-e{}'.format(batch_size, epochs)
log_folder_path = os.path.join('logs',runtype_name + '-{}'.format(run_date))

The Keras Embedding Layer expects a maximum vocabulary size, which we can simply calculate by finding max() of the encoded data

In [14]:
all_data = pd.concat([cic_train_data, cic_test_data])

In [15]:
voc_size = (all_data.max().max()+1).astype('int64')
print("Maximum vocabulary size:", voc_size)

Maximum vocabulary size: 2


## Building and Training the Model

In [16]:
from keras.callbacks import EarlyStopping,ModelCheckpoint,TensorBoard

callbacks = [
    ModelCheckpoint(
        filepath='models/'+runtype_name+'-{}.h5'.format(run_date),
        monitor='val_loss',   
        save_best_only=True    # Only save one. Only overwrite this one if val_loss has improved
    ),
    TensorBoard(
        log_dir=log_folder_path
    )
]

Using TensorFlow backend.


In [18]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.optimizers import RMSprop
from keras.utils import plot_model

# see https://stackoverflow.com/a/49436133/3864726
# This is especially important in an environment like Jupyter, where the Kernel keeps on running
from keras import backend as K
K.clear_session()


model = Sequential()
model.add(Embedding(voc_size, 12,input_length=6)) 
model.add(LSTM(12, name='LSTMnet'))
model.add(Dense(2, activation='softmax')) # Multiclass classification. For binary, one would use i.e. sigmoid

model.compile(optimizer=RMSprop(lr=0.001),
              loss='sparse_categorical_crossentropy', # Multiclass classification! Binary would be binary_crossentropy
              metrics=['acc'])

model.summary()

history = model.fit(cic_train_data, cic_train_labels,
                    epochs=epochs,
                    batch_size=batch_size,
                    verbose=1,
                    validation_data=(cic_test_data, cic_test_labels),
                    callbacks=callbacks)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 6, 12)             24        
_________________________________________________________________
LSTMnet (LSTM)               (None, 12)                1200      
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 26        
Total params: 1,250
Trainable params: 1,250
Non-trainable params: 0
_________________________________________________________________
Train on 1839982 samples, validate on 990761 samples
Epoch 1/50
 956832/1839982 [==============>...............] - ETA: 2:11 - loss: 0.3396 - acc: 0.8532

KeyboardInterrupt: 